<a href="https://colab.research.google.com/github/Hansa-Hettiarachchi/BERT_Text_Classifier/blob/pytorch_dev/Text_calssification_BERT_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.3 MB/s eta 0:00:00


Load Dataset

In [1]:
!nvidia-smi

Thu Jul 20 08:50:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from datasets import load_dataset
imdb = load_dataset('imdb')

  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
imdb['test'][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

Preprocess

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [5]:
def preprocessing(examples):
  return tokenizer(examples["text"], truncation=True)

In [6]:
tokenized_imdb = imdb.map(preprocessing , batched = True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [7]:
from transformers import DataCollatorWithPadding
data_collector = DataCollatorWithPadding(tokenizer = tokenizer)

Evaluate

In [8]:
import evaluate
accuracy = evaluate.load('accuracy')

In [9]:
import numpy as np

def evaluation(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis = 1)
  return accuracy.compute(predictions=predictions, references= labels)


Train

In [10]:
id_to_label = {0: "NEGATIVE", 1: "POSITIVE"}
label_to_id = {"NEGATIVE": 0, "POSITIVE": 1}

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id_to_label, label2id=label_to_id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
!pip install transformers[torch]

In [13]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
! pip install -U accelerate
! pip install -U transformers

In [15]:
training_args = TrainingArguments(
    output_dir="Text_classification_model_1_pytorch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collector,
    compute_metrics=evaluation,
)

trainer.train()

Cloning https://huggingface.co/Hansaht/Text_classification_model_1_pytorch into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.233600,0.271817,0.903000
2,0.162000,0.239178,0.927680
3,0.097100,0.319111,0.931160
4,0.053500,0.321111,0.933400
5,0.034000,0.349357,0.932920


TrainOutput(global_step=7815, training_loss=0.11928659952838529, metrics={'train_runtime': 7884.582, 'train_samples_per_second': 15.854, 'train_steps_per_second': 0.991, 'total_flos': 1.6394784128794656e+16, 'train_loss': 0.11928659952838529, 'epoch': 5.0})

In [16]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Jul20_08-51-52_ffa444e7db0f/events.out.tfevents.1689843124.ffa444e7db0f.10555.0:   0%|       …

To https://huggingface.co/Hansaht/Text_classification_model_1_pytorch
   809011c..947f41e  main -> main

   809011c..947f41e  main -> main

To https://huggingface.co/Hansaht/Text_classification_model_1_pytorch
   947f41e..8784777  main -> main

   947f41e..8784777  main -> main



'https://huggingface.co/Hansaht/Text_classification_model_1_pytorch/commit/947f41eee6c66d5186eb5a208ab52c704b160b73'

In [21]:
text = "It was a masterpiece"
text2 = 'it was like a piece of crap'

In [22]:
from transformers import pipeline
classifier = pipeline('text-classification' , model = "Hansaht/Text_classification_model_1_pytorch")
classifier(text2)

[{'label': 'NEGATIVE', 'score': 0.9935624599456787}]